# Введение

Нашей целью является создание модели нейросети которая могла бы оценить риск ДТП по выбранному маршруту движения. Как только водитель забронировал автомобиль, сел за руль и выбрал маршрут, система должна оценить уровень риска. Если уровень риска высок, водитель увидит предупреждение и рекомендации по маршруту.

Важная подзадача — понять, возможно ли предсказывать ДТП, опираясь на исторические данные одного из регионов.

Для достижения цели нужно выполнить следующие задачи:
- Провести загрузку данных через запросы SQL;
- Провести предобработку данных;
- Исследовательский анализ данных с изображением графиков;
- Для рабочей группы специалистов сформировать 6 вопросов и ответить на 2 из них графически в том числе;
- Сделать не менее 3-х моделей предсказания с перебором разных гиперпараметрами;
- Выбрать одну лучшую модель;
- Оценить вклад каждого признака в модели в ДТП;
- На основе модели исследовать основные факторы ДТП;
- Понять, помогут ли результаты моделирования и анализ важности факторов ответить на вопросы:
    1. Возможно ли создать адекватную системы оценки водительского риска при выдаче авто?
    2. Какие ещё факторы нужно учесть?
    3. Нужно ли оборудовать автомобиль какими-либо датчиками или камерой?
- Сформировать для заказчика вывод, написав рекомендации.

# Подключение к базе. Загрузка таблицы sql

Подгружаем необходимые модули и библиотеки.

In [ ]:
#!pip install sqlalchemy
#!pip install --upgrade scikit-learn
#!pip install shap
#!pip install phik
#!pip install skorch

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import seaborn as sns
from scipy import stats as st
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import sklearn
import random
import shap
import phik
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, RobustScaler, TargetEncoder, OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sqlalchemy import create_engine 
from sqlalchemy.sql import select
from torch.utils.data import DataLoader, TensorDataset
from itertools import product
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, 
                             recall_score, 
                             precision_score, 
                             confusion_matrix, 
                             roc_auc_score, 
                             classification_report, 
                             f1_score, 
                             precision_recall_curve, 
                             average_precision_score, 
                             RocCurveDisplay)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from skorch import NeuralNetClassifier
from tabulate import tabulate
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold

Создаем подключение к серверу и подключаемся.

In [ ]:
db_config = {
'user': 'praktikum_student',
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432,
'db': 'data-science-vehicle-db'
} 

In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 

In [ ]:
engine = create_engine(connection_string)

Загружаем таблицы SQL в переменные через запросы БД.

Посмотрим на наличие таблиц в базе.

In [ ]:
required_tables = ['collisions', 'parties', 'vehicles', 'case_ids']
def check_tables_exist(tables_list):

    existing_tables = pd.read_sql("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """, engine)['table_name'].tolist()

    
    results = {}
    for table in tables_list:
        results[table] = table in existing_tables
    
    return results


table_status = check_tables_exist(required_tables)
display(table_status)

## Осмотр данных

Теперь взглянем на таблицы.

- case_ids - объединяющая таблица
- collisions — общая информация о ДТП
- parties — информация об участниках ДТП
- vehicles — информация о пострадавших машинах

In [ ]:
def table(table_name, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit}"
    df = pd.read_sql(query, engine)
    display(df)
    return df

case_ids_df = table('case_ids')
vehicles_df = table('vehicles')
collisions_df = table('collisions')
parties_df = table('parties')

Все таблицы связаны одним общим ключом - case_id и соединены общей таблицей case_ids.
Выгрузка произошла корректно, данные в порядке. Приступим к следующему этапу - обработка.

## Первичное исследование таблиц

Загрузим нужную информацию и посмотрим на общее число значений ДТП.

Значение car присутствует, что говорит о наличии нужных нам данных. Так же как и возраст авто.

Данные в наличии, имеется общая таблица через которую можно объединить все, по case_id. Можно приступать к анализу.

#  Статистический анализ факторов ДТП

Посмотрим на совершенные ДТП по месяцам.

In [ ]:
col_count = """
SELECT 
    EXTRACT(MONTH FROM collision_date) AS month,
    COUNT(case_id) AS accidents_count
FROM 
    collisions
GROUP BY 
    EXTRACT(MONTH FROM collision_date)
ORDER BY 
    accidents_count DESC;
"""
col_count = pd.read_sql_query(col_count, con = engine)
col_count

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(col_count['month'], col_count['accidents_count'], color='skyblue')
plt.title('Количество ДТП по месяцам', fontsize=14)
plt.xlabel('Месяц', fontsize=12)
plt.ylabel('Количество аварий', fontsize=12)
plt.xticks(col_count['month'], 
          ['Янв', 'Фев', 'Мар', 'Апр', 'Май', 'Июн', 
           'Июл', 'Авг', 'Сен', 'Окт', 'Ноя', 'Дек'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Согласно графика, мы имеем большее количество ДТП в зимне-осенний период. Затем летне-осенний период снижение. И опять рост к декабрю. В целом, это нормальное явление при таких погодных условиях.

Скоро состоится первое совещание рабочей группы. Чтобы обсуждение было конструктивным, каждый сотрудник должен понимать данные. Для этого создадим подходящие аналитические задачи.

Список задач:
1. как часто происходят нарушения в дневное время (A — Daylight) суток в алкогольном опьянении B — Had Been Drinking, Under Influence с влиянием в los angeles
2. Влияет ли возраст автомобиля VEHICLE_AGE на дтп с результатом в виде восстановлению не подлежит 1 — FATAL ТС
3. С каким наибольшим по количеству типом участника на перекрестках Y — Intersection происходят дтп?
4. Какие типы повреждений являются самыми убыточными для страховой компании с участниками в алкогольном опьянении B — Had Been Drinking, Under Influence
5. Где больше всего повреждают дорожные знаки?
6. На каком дорожном покрытии происходят ДТП с наименьшим количеством участников?

Для наглядности коллегам решим две задачи из списка, вторую и пятую.

In [ ]:
vehicle_age = """
SELECT COUNT(v.vehicle_age), v.vehicle_age
FROM vehicles v
JOIN case_ids cids ON cids.case_id = v.case_id
JOIN collisions c ON c.case_id = cids.case_id
WHERE collision_damage = 'fatal'
GROUP BY collision_damage, vehicle_age
ORDER BY v.vehicle_age
"""
vehicle_age = pd.read_sql_query(vehicle_age, con = engine)
vehicle_age

Есть ошибочное значение в 161 год. Далее мы исправим это.

In [ ]:
vehicle_age['count'].plot(figsize=(10, 6))
plt.title('Зависимость числа ДТП типа fatal от возраста авто')
plt.xlabel('Возраст')
plt.ylabel('Число ДТП')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
sings = """
SELECT COUNT(party_type), county_location
FROM collisions c
JOIN case_ids cids ON cids.case_id = c.case_id
JOIN parties p ON c.case_id = p.case_id
WHERE p.party_type = 'road signs'
GROUP BY county_location
ORDER BY COUNT(party_type) DESC
LIMIT 10
"""

sings = pd.read_sql_query(sings, con = engine)
sings

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(sings['county_location'], sings['count'], color='orange')
plt.title('Место частых ДТП', fontsize=16)
plt.xlabel('Локация ДТП', fontsize=12)
plt.ylabel('Количество аварий', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Больше всего данных ДТП происходит в Лос-Анджелесе.

# Создание модели для оценки водительского риска

### Выгрузка данных в общий датафрейм

Время сделать общий датафрейм и начать работу над моделями.

In [ ]:
df = """
SELECT *
FROM collisions c
JOIN case_ids cids 
    ON c.case_id = cids.case_id
JOIN parties p 
    ON c.case_id = p.case_id
JOIN vehicles v 
    ON c.case_id = v.case_id 
    AND p.party_number = v.party_number
WHERE p.party_type = 'car' 
    AND EXTRACT(YEAR FROM c.collision_date) = 2012 
    AND c.collision_damage != 'scratch'
"""

df = pd.read_sql_query(df, con=engine)
df_net_3 = df.copy()
df.head(3)

In [ ]:
pd.set_option('display.max_columns', None)
display(df.head(1))

Проверим, все ли связи между таблицами учтены.

In [ ]:
null_check = df[['case_id']].isnull().sum()
print("Пропущенные значения в ключевых полях:")
display(null_check)

In [ ]:
relations_to_check = [
    ("parties-case_ids", "SELECT COUNT(*) FROM parties p LEFT JOIN case_ids c ON p.case_id = c.case_id WHERE c.case_id IS NULL"),
    ("case_ids-collisions", "SELECT COUNT(*) FROM case_ids c LEFT JOIN collisions col ON c.case_id = col.case_id WHERE col.case_id IS NULL AND EXTRACT(YEAR FROM col.collision_date) = 2012 AND col.collision_damage != 'scratch'"),
    ("collisions-vehicles", "SELECT COUNT(*) FROM collisions col LEFT JOIN vehicles v ON col.case_id = v.case_id WHERE v.case_id IS NULL AND EXTRACT(YEAR FROM col.collision_date) = 2012 AND col.collision_damage != 'scratch'")
]

for name, query in relations_to_check:
    result = pd.read_sql_query(query, engine)
    print(f"Потерянные записи при связи {name}: {result.iloc[0,0]}")

65536 значений отфильтровано по условиям задачи.
Выгрузка прошла хорошо, все отображается. Проведем первичный отбор факторов, необходимых для модели. Нужно отобрать те, которые могут влиять на вероятность ДТП. Прокомментируем каждый выбранный признак.

- **cellphone_in_use**. Наличие телефона без помощи рук подразумевает его использование. Независимо от функции разговора без помощи рук - это фактор отвлечения от дороги и риск совершения ДТП.
- **vehicle_age**. Возраст автомобиля напрямую влияет на его техническое состояние а значит на поломку в момент движения и создания опасной ситуации на дороге.
- **distance**. Дистанция от главной дороги не является одни из определяющих факторов ДТП, но если движения осуществлялось по главной дороге (значение 0.0), то значит движение более интенсивное.
- **party_sobriety**. Алкогольное опьянение безусловно влияет на риск совершения ДТП.
- **party_drug_physical**. Наркотическое опьянение также влияет на риск совершения ДТП.
- **intersection**. Перекресток одно из самых сложных и опасных частей дорог. Регулируемые и нерегулируемые по своему создают сложности водителям, где нужно быть максимально осторожным. 
- **weather_1**. Погодные условия один из ключевых факторов события ДТП. Особенно если это туман и снегопад.
- **location_type**. Тип дорог обуславливает сложность правил и интенсивность потока. Чем больше полос, скорость и режимов поворота - тем опаснее дорога.
- **primary_collision_factor**. Основной фактор аварии это то, что приводит к ДТП непосредственно от водителя. Нарушение правил и сон за рулем уже практически совершенное ДТП.
- **pcf_violation_category**. Так же как и предыдущий пункт, этот конкретизирует вид нарушения.
- **motor_vehicle_involved_with**. Дополнительные участники ДТП означает что количество объектов вокруг водителя совершившего ДТП для него являлось одним из элементов ДТП. Не будь рядом машины, мотоцикла, знака, отбойника, в некоторых случая бы обошлось без ДТП.
- **road_surface**. Состояние дороги критически важный элемент при совершении ДТП.
- **road_condition_1**. Состояние дороги также оказывает одно из ключевых влияний на риск совершения ДТП. 
- **lighting**. Освещенность само собой тоже главный элемент.
- **control_device**. Поврежденные приборы контроля могут повысить риск совершения ДТП.

Возможно, придеться что-то добавить или убрать из этого списка признаков при построении моделей.

Остальные отбросим.

In [ ]:
columns = ['county_city_location', 
           'cellphone_in_use',
           'county_location',
           'vehicle_age', 
           'weather_1',
          'control_device', 
          'at_fault']

df = df[columns]

Теперь удалим пустые значения, как и планировали.

## Статистическое исследование отобранных признаков

Откроем общие данные датасета и взглянем на статистику.

In [ ]:
df.info()

In [ ]:
display(round(df.describe(), 3))
display(df.median())

In [ ]:
df.median()

Все признаки в норме, кроме одного выброса в 161 год возраста.

Построим графики на каждый признак.

In [ ]:
plt.title('График возраста автомобиля', fontsize=16)
plt.ylabel('Количество авто', fontsize=14)
plt.xlabel('Возраст', fontsize=14)
df['vehicle_age'].hist(bins=10);

In [ ]:
config = {
    'cellphone_in_use': ['Наличие телефона в автомобиле', 'Наличие', 'Число оснащенных авто'], 
    'party_sobriety': ['Трезвость участника', 'Уровень трезвости', 'Количество лиц'],
    'party_drug_physical': ['Состояние участника: физическое или с учётом принятых лекарств', 'Уровень трезвости', 'Количество лиц'],
    'weather_1': ['Погода', 'Тип', 'Количество случаев'],
    'control_device': ['Прибор контроля', 'Наличие', 'Число оснащенных авто'],
    'at_fault': ['Целевая переменная', 'Факт', 'Количество']
}

for column, label in config.items():
    random_color = np.random.rand(3,)
    ax = df[column].value_counts().plot.bar(
        color=random_color,
        stacked=True,
        grid=True,
        rot=45,
        figsize=(10, 5),
        title=label[0]
    )
    ax.set_ylabel(label[2], fontsize=14)
    ax.set_xlabel(label[1], fontsize=14)
    plt.tight_layout()
    plt.show()


## Корреляционный анализ

Построим матрицу корреляции.

In [ ]:
df['county_city_location'] = df['county_city_location'].astype('int')

In [ ]:
auto = {'cellphone_in_use':'binary', 
               'vehicle_age':'interval',  
               'distance':'interval', 
               'party_sobriety':'categorical', 
               'party_drug_physical':'categorical', 
               'intersection':'binary', 
               'weather_1':'categorical', 
               'location_type':'categorical',
               'primary_collision_factor':'categorical', 
               'pcf_violation_category':'categorical', 
               'motor_vehicle_involved_with':'categorical',
               'road_surface':'categorical', 
               'road_condition_1':'categorical', 
               'lighting':'categorical', 
               'control_device':'binary',
               'county_city_location': 'interval'
              }

interval_cols = [col for col, col_type in auto.items() if col_type == 'interval']

corr = df.phik_matrix(interval_cols=interval_cols)

figsize = (14,14)
fig, ax = plt.subplots(figsize=figsize)
plt.title('Матрица корреляции Фи'+'\n',size=(16))

sns.heatmap(corr, ax=ax, annot=True, fmt='.2f', cmap="RdBu_r", square=True) ;

Есть немого хороших зависимостей, например **PCF_VIOLATION_CATEGORY** с **INTERSECTION**, 0.78 и с **PARTY_SOBRIETY** 0.58.

**MOTOR_VEHICLE_INVOLVED_WITH** имеет с **PCF_VIOLATION_CATEGORY** 0.66

**ROAD_SURFACE c WEATHER_1** - 0.69

И небольшое влияние наркотического опьянения **PARTY_DRUG_PHYSICAL** на ДТП (**AT_FAULT**) 0.46.

## Вывод по зависимости и вкладам признаков

Не пил - является самым многочисленным по причине ДТП. Соответственно, наша модель верна.

**Трезвость** Наличие нужного оборудования для выявления статуса трезвости участника ДТП является сильно сложным с технологической точки зрения но выполнимым. Самым простым решением будет проверка первых 20 секунд стиля вождения автомобиля. Если компьютер выявляет неадекватные от общей нормы (собранная статистика по личному авто этого водителя) манеры вождения - авто останавливается и блокирует двигатель, по возможности вызывая инспектора полиции. Кроме того, перед поездкой при наличии подозрения или признаков опьянения компания имеет право не выдавать автомобиль. Все подозрения следует визуально фиксировать в случае юридических споров.

**Возраст авто** Можно сказать что для предотвращения ДТП от длительности эксплуатации спасает обычное техническое обслуживание и проверка. При любом нештатном состоянии следует исправлять и машину клиенту не выдавать.

**Контрольное оборудование** Аналогично предыдущему. Контрольное оборудование необходимо проверять на его исправность и корректность выдаваемых данных устройством.

**Регион где произошло ДТП** Путем статистического анализа, который может собирать каршеринговая компания по регионам ее работы, можно сделать вывод какие регионы опасны, например для малоопытных водителей, или вообще опасны. Где риск вероятного ДТП велик и какие факторы в этом замешаны именно в данных регионах. Например, регионы с извилистым серпантином или больше города с интенсивных движением повышают вероятность ДТП.

**Наркотическое опьянение** Полностью аналогично первому пункту. 

Остальные зависимости крайне незначительны. 

## Построение моделей

Начнем создавать модели. Сделаем 3 разных архитектуры с перебором разных параметров у каждой модели. Подготовим данные для моделей. Возьмем енкодер TargetEncoder, масштабировать будем StandardScaler. В конце выборки преобразуем в тензоры.

In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.use_deterministic_algorithms(True)

encoder = TargetEncoder(
    target_type='binary',
    smooth=10.0,
    cv=5,
    random_state=42
)

scaler = StandardScaler()

num = ['vehicle_age']
cat = ['weather_1', 'control_device', 'county_city_location', 'county_location']
cat_not_cod = ['cellphone_in_use']
cat_cols_to_encode = [col for col in cat if col not in cat_not_cod]
nan_to_check = ['control_device', 'weather_1']

df_train, df_test = train_test_split(df, test_size=0.25, random_state=2025) 
X_train = df_train.drop(['at_fault'], axis=1)
y_train = df_train['at_fault']
X_test = df_test.drop(['at_fault'], axis=1)
y_test = df_test['at_fault']

In [ ]:
X_train['vehicle_age'] = X_train['vehicle_age'].replace(161, X_train['vehicle_age'].mode()[0])

In [ ]:
fill_values = {
    'party_drug_physical': X_train['party_drug_physical'].mode()[0],
    'party_sobriety': 'not applicable',
    'cellphone_in_use': 0,
    'county_city_location': 0,
    'county_location': 'uknown',
    'vehicle_age': 0,
    'weather_1': 'clear',
    'control_device': 'functioning'
}

### Первая модель

Возьмем 5 слоев, 128, 64, 32, 16, 1 нейронов соответственно. Во входящем слое используем регуляризацию BatchNorm1d. Функции активации везде будут LeakyReLU. Так как бинарная классификация функция активации на выходном слое Sigmoid. Обучаем в 100 эпох.

In [ ]:

class Net(nn.Module):
    def __init__(self, input_size: int, learning_rate, h_neurons=(128, 64, 32, 16, 1)):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, h_neurons[0]),
            nn.BatchNorm1d(h_neurons[0]),
            nn.LeakyReLU(),
            
            nn.Linear(h_neurons[0], h_neurons[1]),
            nn.BatchNorm1d(h_neurons[1]),
            nn.LeakyReLU(),        
            
            nn.Linear(h_neurons[1], h_neurons[2]),
            nn.BatchNorm1d(h_neurons[2]),
            nn.LeakyReLU(),                  
            
            nn.Linear(h_neurons[2], h_neurons[3]),
            nn.LeakyReLU(),
                        
            nn.Linear(h_neurons[3], h_neurons[4])
        )
    def forward(self, x):
        return self.net(x).squeeze(-1)


Для перебора будем брать 128 и 256 размер батча и три значения learning_rate. Оптимизатор Adam, функция потерь BCEWithLogitsLos. Метрику используем ROC_AUC как самую оптимальную для бинарной классификации с учетом ее универсальности, не зависит от порога классификации. Кроме того, учитывает и Recall (TPR), и Specificity (1-FPR).

In [ ]:
batch_sizes = [128, 256]
learning_rates = [1e-4, 1e-5]
num_epochs = 100
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_auc = 0
best_model = None
best_params = None

for col, value in fill_values.items():
    X_train[col] = X_train[col].fillna(value)
    X_test[col] = X_test[col].fillna(value)
    
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', encoder)
])

data_preprocessor = ColumnTransformer(
    transformers=[
        ('cat_processor', cat_pipeline, cat_cols_to_encode), 
        ('num', StandardScaler(), num)
    ],
    remainder='passthrough'
)

X_train_processed = data_preprocessor.fit_transform(X_train, y_train)
X_train_processed = np.asarray(X_train_processed, dtype=np.float32)

for b, lr in product(batch_sizes, learning_rates):
    print(f"\n=== Тест batch_size={b}, learning_rate={lr} ===")
    fold_aucs = []

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_processed, y_train)):
        print(f"\n--- Фолд {fold+1} ---")
        X_tr = X_train_processed[train_idx]
        y_tr = y_train.iloc[train_idx].values
        X_val = X_train_processed[valid_idx]
        y_val = y_train.iloc[valid_idx].values
        train_ds = TensorDataset(torch.FloatTensor(X_tr), torch.FloatTensor(y_tr))
        train_loader = DataLoader(train_ds, batch_size=b, shuffle=True, pin_memory=True)

        model = Net(input_size=X_tr.shape[1], learning_rate=lr)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()

        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            preds_val = model(torch.FloatTensor(X_val))
            roc_auc = roc_auc_score(y_val, preds_val.numpy())
            fold_aucs.append(roc_auc)
            print(f"Фолд {fold+1} ROC-AUC: {roc_auc:.4f}")

    avg_auc = np.mean(fold_aucs)
    std_roc_auc = np.std(fold_aucs)
    print(f"\nСредняя ROC-AUC для batch_size={b}, learning_rate={lr}: {avg_auc:.4f}")

    if avg_auc > best_auc:
        best_auc = avg_auc
        best_model = model.state_dict()
        best_params = {'learning_rate': lr, 'batch_size': b}

X_test_processed = data_preprocessor.transform(X_test)
X_test_processed = np.asarray(X_test_processed, dtype=np.float32)

print(f"\nЛучшие параметры: Learning rate={best_params['learning_rate']}, Batch Size={best_params['batch_size']}")
print(f"Лучшая КВ ROC-AUC: {best_auc:.4f}")


ROC-AUC показывает 0.68. Неплохой результат, но не совершенный.

### Вторая модель

Теперь используем дропаут (0.0, 0.1) в третьем слое и количество нейронов в слоях 256, 128, 64, 1. Функция активации в первом LeakyReLU, в третьем ReLU. Размер батчей оставим прежним. Уберем learnin_rate. 100 эпох.

In [ ]:
class Net_2(nn.Module):
    def __init__(self, input_size: int, dropout:float, h_neurons=(256, 128, 64, 1)):
        super(Net_2, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, h_neurons[0]),
            nn.BatchNorm1d(h_neurons[0]),
            nn.LeakyReLU(),
            
            nn.Linear(h_neurons[0], h_neurons[1]),
            nn.ReLU(), 
            nn.Dropout(dropout),
            
            nn.Linear(h_neurons[1], h_neurons[2]),
            nn.ReLU(),
            nn.Dropout(dropout),                  
                        
            nn.Linear(h_neurons[2], h_neurons[3])
        )
    def forward(self, x):
        return self.net(x).squeeze(-1) 
    
dropout = [0.0, 0.1]
batch_size = [128, 256]
results = {}

num_epochs = 100

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_auc_2 = 0
best_model_2 = None
best_params_2 = None

for b, d in product(batch_sizes, dropout):
    print(f"\n=== Тест batch_size={b}, dropout={d} ===")
    fold_aucs = []

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"\n--- Фолд {fold+1} ---")
        X_train_2, X_valid_2 = X_train.iloc[train_idx].copy(), X_train.iloc[valid_idx].copy()
        y_train_2, y_valid_2 = y_train.iloc[train_idx].copy(), y_train.iloc[valid_idx].copy()

        cat_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', encoder)
        ])
        data_preprocessor = ColumnTransformer(
            transformers=[
                ('cat_processor', cat_pipeline, cat_cols_to_encode), 
                ('num', StandardScaler(), num)
            ],
            remainder='passthrough'
        )

        X_train_enc = data_preprocessor.fit_transform(X_train_2, y_train_2)
        X_valid_enc = data_preprocessor.transform(X_valid_2)
        X_train_enc = np.asarray(X_train_enc, dtype=np.float32)
        X_valid_enc = np.asarray(X_valid_enc, dtype=np.float32)

        X_train_tensor = torch.FloatTensor(X_train_enc)
        y_train_tensor = torch.FloatTensor(y_train_2.values)
        X_valid_tensor = torch.FloatTensor(X_valid_enc)
        y_valid_tensor = torch.FloatTensor(y_valid_2.values)

        train_ds = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_ds, batch_size=b, shuffle=True, pin_memory=True)

        model_2 = Net_2(input_size=X_train_tensor.shape[1], dropout=d)
        optimizer = optim.Adam(model_2.parameters(), lr=1e-4)
        criterion = nn.BCEWithLogitsLoss()

        model_2.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                preds = model_2(X_batch)
                loss = criterion(preds, y_batch)
                loss.backward()
                optimizer.step()

        model_2.eval()
        with torch.no_grad():
            preds_valid = model_2(X_valid_tensor)
            roc_auc = roc_auc_score(y_valid_tensor.numpy(), preds_valid.numpy())
            fold_aucs.append(roc_auc)
            print(f"Фолд {fold+1} ROC-AUC: {roc_auc:.4f}")

    avg_auc = np.mean(fold_aucs)
    std_roc_auc_2 = np.std(fold_aucs)
    print(f"\nСредняя ROC-AUC for batch_size={b}, dropout={d}: {avg_auc:.4f}")

    if avg_auc > best_auc_2:
        best_auc_2 = avg_auc
        best_model_2 = model_2.state_dict()
        best_params_2 = {'dropout': d, 'batch_size': b}

print(f"\nЛучшие параметры: Dropout={best_params_2['dropout']}, Batch Size={best_params_2['batch_size']}")
print(f"Лучшая КВ ROC-AUC: {best_auc_2:.4f}")


Метрика поднялась, 0.68. Но все еще недостаточно.

### Третья модель

In [ ]:
results = {}
best_roc_auc_3 = 0
best_model_3 = None
best_params_3 = None
best_preds_3 = None

solver_list = ['lbfgs', 'liblinear', 'saga']
max_iter_list = [50, 100, 150]
penalty_list = ['l2', None] 


X_train_3, X_valid_3, y_train_3, y_valid_3 = train_test_split(
    X_train, y_train,
    test_size=0.3, 
    random_state=42,
    stratify=y_train
)

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', encoder)
])

data_preprocessor = ColumnTransformer(
    transformers=[
        ('cat_processor', cat_pipeline, cat_cols_to_encode), 
        ('num', StandardScaler(), num)
    ],
    remainder='passthrough'
)


X_train_3 = data_preprocessor.fit_transform(X_train_3, y_train_3)
X_valid_3 = data_preprocessor.transform(X_valid_3)
    
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for solver in solver_list:
    for max_iter in max_iter_list:
        for penalty in penalty_list:
            if solver == 'liblinear' and penalty is None:
                continue
            if solver in ['newton-cg', 'sag', 'lbfgs'] and penalty not in [None, 'l2']:
                continue
                
            try:
                model_3 = LogisticRegression(
                    solver=solver,
                    max_iter=max_iter,
                    penalty=penalty,
                    random_state=42,
                    n_jobs=-1
                )
                
                cv_scores = cross_val_score(
                    estimator=model_3,
                    X=X_train_3,
                    y=y_train_3,
                    cv=cv,
                    scoring='roc_auc',
                    n_jobs=-1
                )
                
                mean_roc_auc = np.mean(cv_scores)
                std_roc_auc_3 = np.std(cv_scores)
                
                print(f"Solver: {solver:<9} | Max_iter: {max_iter:<4} | Penalty: {str(penalty):<4} | "
                      f"ROC-AUC: {mean_roc_auc:.4f} ± {std_roc_auc_3:.4f}")
                
                if mean_roc_auc > best_roc_auc_3:
                    best_roc_auc_3 = mean_roc_auc
                    best_model_3 = model_3.fit(X_train_3, y_train_3)
                    best_params_3 = {
                        'solver': solver,
                        'max_iter': max_iter,
                        'penalty': penalty
                    }
                    
            except Exception as e:
                print(f"Ошибка для комбинации solver={solver}, max_iter={max_iter}, penalty={penalty}: {str(e)}")
                continue

                
if best_model_3 is not None:
    best_preds_3 = best_model_3.predict(X_valid_3)
    final_roc_auc_3 = roc_auc_score(y_valid_3, best_preds_3)
    
    print(f"\nЛучшая конфигурация: "
          f"Solver={best_params_3['solver']}, "
          f"Max_iter={best_params_3['max_iter']}, "
          f"Penalty={best_params_3['penalty']}")
    print(f"Средний ROC-AUC на кросс-валидации: {best_roc_auc_3:.4f}")
    print(f"ROC-AUC на валидационном наборе: {final_roc_auc_3:.4f}")

0.61, чтож это пока лучший результат у этой модели. Выше поднять не удалось.

### Четвертая модель

In [ ]:
results = {}
best_roc_auc_4 = 0
best_model_4 = None
best_params_4 = None
best_preds_4 = None

n_estimators_list = [50, 80, 150]
max_depth_list = [3, 5, None]
min_samples_leaf_list = [1, 2, 3]
max_features_list = ['sqrt', 'log2']

X_train_4, X_valid_4, y_train_4, y_valid_4 = train_test_split(
    X_train, y_train, # то же что и в предыдущей
    test_size=0.3, 
    random_state=42,
    stratify=y_train
)
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', encoder)
])

data_preprocessor = ColumnTransformer(
    transformers=[
        ('cat_processor', cat_pipeline, cat_cols_to_encode), 
        ('num', StandardScaler(), num)
    ],
    remainder='passthrough'
)


X_train_4 = data_preprocessor.fit_transform(X_train_4, y_train_4)
X_valid_4 = data_preprocessor.transform(X_valid_4)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for min_samples_leaf in min_samples_leaf_list:
            for max_features in max_features_list:
                try:
                    model_4 = RandomForestClassifier(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_leaf=min_samples_leaf,
                        max_features=max_features,
                        random_state=42,
                        n_jobs=-1
                    )
                    

                    cv_scores = cross_val_score(
                        estimator=model_4,
                        X=X_train_4,
                        y=y_train_4,
                        cv=cv,
                        scoring='roc_auc',
                        n_jobs=1
                    )
                    
                    mean_roc_auc = np.mean(cv_scores)
                    std_roc_auc_4 = np.std(cv_scores)
                    
                    print(f"N_est: {n_estimators:<3} | Max_d: {str(max_depth):<4} | "
                          f"Min_leaf: {min_samples_leaf:<2} | Max_feat: {max_features:<5} | "
                          f"ROC-AUC: {mean_roc_auc:.4f} ± {std_roc_auc_4:.4f}")
                    

                    if mean_roc_auc > best_roc_auc_4:
                        best_roc_auc_4 = mean_roc_auc
                        best_model_4 = model_4.fit(X_train_4, y_train_4)
                        best_params_4 = {
                            'n_estimators': n_estimators,
                            'max_depth': max_depth,
                            'min_samples_leaf': min_samples_leaf,
                            'max_features': max_features
                        }
                        
                except Exception as e:
                    print(f"Ошибка для комбинации n_est={n_estimators}, max_d={max_depth}, "
                          f"min_leaf={min_samples_leaf}, max_feat={max_features}: {str(e)}")
                    continue

if best_model_4 is not None:
    best_preds_4 = best_model_4.predict(X_valid_4)
    final_roc_auc_4 = roc_auc_score(y_valid_4, best_preds_4)
    
    print(f"\nЛучшая конфигурация: "
          f"N_estimators={best_params_4['n_estimators']}, "
          f"Max_depth={best_params_4['max_depth']}, "
          f"Min_samples_leaf={best_params_4['min_samples_leaf']}, "
          f"Max_features={best_params_4['max_features']}")
    print(f"Средний ROC-AUC на кросс-валидации: {best_roc_auc_4:.4f}")
    print(f"ROC-AUC на валидационном наборе: {final_roc_auc_4:.4f}")

### Вывод по созданию моделей

Выведем всю статистику обучения и результаты на графики.

In [ ]:
results = {"Net": [best_auc, std_roc_auc], 
           "Net_2": [best_auc_2, std_roc_auc_2], 
           "Logistic Regression": [final_roc_auc_3, std_roc_auc_3],
           "Random Forest": [final_roc_auc_4, std_roc_auc_4],
}

In [ ]:
df_results = pd.DataFrame(results, index=["Средняя AUC", "Std"]).T
df_results.sort_values("Средняя AUC", ascending=False, inplace=True)

plt.figure(figsize=(12, 7))
bar_plot = plt.barh(df_results.index, df_results["Средняя AUC"], 
                   xerr=df_results["Std"], 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
                   alpha=0.7,
                   capsize=5)

In [ ]:
comparison_data = {
    "Модель": ["Net (Нейросеть)", "Net_2 (Нейросеть 2)", "Logistic Regression", "Random Forest"],
    "Лучшая ROC-AUC": [best_auc, best_auc_2, final_roc_auc_3, final_roc_auc_4],
    "Std": [std_roc_auc, std_roc_auc_2, std_roc_auc_3, std_roc_auc_4],
    "Параметры": [
        f"batch_size={best_params['batch_size']}, lr={best_params['learning_rate']}",
        f"batch_size={best_params_2['batch_size']}, dropout={best_params_2['dropout']}",
        f"solver={best_params_3['solver']}, max_iter={best_params_3['max_iter']}, penalty={best_params_3.get('penalty', 'None')}",
        f"n_est={best_params_4['n_estimators']}, max_d={best_params_4['max_depth']}, min_leaf={best_params_4['min_samples_leaf']}, max_feat={best_params_4['max_features']}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values("Лучшая ROC-AUC", ascending=False)

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

styled_table = (comparison_df.style
               .format({
                   "Лучшая ROC-AUC": "{:.4f}",
                   "Std": "{:.4f}"
               })
               .apply(highlight_max, subset=["Лучшая ROC-AUC"])
               .set_caption("Сравнение производительности моделей"))


display(styled_table)

Применим Net к тестовым данным.

In [ ]:
fill_values = {
    'party_drug_physical': X_test['party_drug_physical'].mode()[0] if not X_test['party_drug_physical'].mode().empty else 'unknown',
    'party_sobriety': 'not applicable',
    'cellphone_in_use': 0,
    'county_city_location': 0,
    'county_location': 'unknown',
    'vehicle_age': 0,
    'weather_1': 'clear',
    'control_device': 'functioning'
}

for col, value in fill_values.items():
    if col in X_test.columns:
        X_test[col] = X_test[col].fillna(value)

X_test_processed = data_preprocessor.transform(X_test)
X_test_processed = np.asarray(X_test_processed, dtype=np.float32)

if not hasattr(model, 'state_dict'):
    final_model = Net(input_size=X_test_processed.shape[1], learning_rate=best_params['learning_rate'])
    final_model.load_state_dict(best_model)
else:
    final_model = model

final_model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_processed)
    y_test_pred_proba = final_model(X_test_tensor).numpy().squeeze()
    y_test_pred = (y_test_pred_proba > 0.5).astype(int)

try:
    test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
    print(f"\nФинальная оценка на тестовых данных:")
    print(f"ROC-AUC: {test_roc_auc:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_test_pred))
    
except Exception as e:
    print(f"Ошибка вычисления метрик: {str(e)}")

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Предсказано 0', 'Предсказано 1'],
            yticklabels=['Фактически 0', 'Фактически 1'])
plt.title('Матрица ошибок Net')
plt.ylabel('Фактические значения')
plt.xlabel('Предсказанные значения')
plt.show()

tn, fp, fn, tp = cm.ravel()

metrics = {
    "Метрика": [
        "Точность (Precision)", 
        "Полнота (Recall)", 
        "F1-мера",
        "Аккуратность (Accuracy)",
        "Специфичность",
        "Ложноположительная ставка"
    ],
    "Значение": [
        precision_score(y_test, y_test_pred),
        recall_score(y_test, y_test_pred),
        f1_score(y_test, y_test_pred),
        accuracy_score(y_test, y_test_pred),
        tn / (tn + fp),
        fp / (fp + tn)
    ],
    "Бизнес-интерпретация": [
        "Какая доля предсказанных '1' действительно верна",
        "Какая доля реальных '1' была выявлена",
        "Баланс между точностью и полнотой",
        "Общая доля верных предсказаний",
        "Способность находить отрицательные случаи",
        "Доля ложных тревог"
    ]
}

metrics_df = pd.DataFrame(metrics)
print("\nКлючевые метрики модели:")
print(tabulate(metrics_df, headers='keys', tablefmt='pretty', showindex=False))

Модель предсказала 6610 случаев когда ДТП не случится и оно не случилось.
5201 было предсказано как ошибочное, она указала как ДТП не произойдет, а на самом деле ДТП произошло.
1890 модель предсказала что ДТП произойдет и оно произошло.
и Всего 361 ДТП было предсказано, но оно не произошло.

83% предсказанных ДТП действительно произошли и всего 26% было выявлено. Среднее между ними составляет 40%.

Всего модель верно попала в 60% случаев.

Модель великолепно находит случаи, когда ДТП не произошло.

Ложных срабатываний всего 5%.

# Проведите анализ важности факторов ДТП

Посмотрим теперь на вклад признаков в обучение модели и на самый важный фактор влияющий на ДТП через график значений SHAP.

In [ ]:
def get_feature_names_from_preprocessor(preprocessor, input_df):
    output_features = []

    for name, transformer, cols in preprocessor.transformers_:
        if transformer == 'drop':
            continue

        elif transformer == 'passthrough':
            if isinstance(cols, slice):
                passthrough_cols = input_df.columns[cols]
            else:
                passthrough_cols = cols
            output_features.extend(passthrough_cols)

        elif isinstance(transformer, Pipeline):
            last_step = transformer.steps[-1][1]
            if hasattr(last_step, 'get_feature_names_out'):
                names = last_step.get_feature_names_out(cols)
                output_features.extend(names)
            else:
                if isinstance(cols, (list, tuple, np.ndarray)):
                    output_features.extend(cols)
                else:
                    output_features.append(cols)

        elif hasattr(transformer, 'get_feature_names_out'):
            names = transformer.get_feature_names_out(cols)
            output_features.extend(names)

        else:
            if isinstance(cols, (list, tuple, np.ndarray)):
                output_features.extend(cols)
            else:
                output_features.append(cols)

    return output_features

X_train_processed = data_preprocessor.fit_transform(X_train, y_train)
X_test_processed = data_preprocessor.transform(X_test)
feature_names = cat_cols_to_encode + num + cat_not_cod
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names)

def analyze_top_shap_features(model, X_test_processed, n_top=5):
    assert isinstance(model, torch.nn.Module)

    if hasattr(X_test_processed, 'columns'):
        feature_names = X_test_processed.columns.tolist()
        X_test_values = X_test_processed.values
    else:
        feature_names = [f'Feature_{i}' for i in range(X_test_processed.shape[1])]
        X_test_values = X_test_processed
        
    def model_predict(x):
        model.eval()
        with torch.no_grad():
            x_tensor = torch.FloatTensor(x)
            return model(x_tensor).numpy()
    
    try:
        sample_size = min(1000, len(X_test_values))
        X_sample = X_test_values[:sample_size]
        
        explainer = shap.Explainer(
            model_predict, 
            X_sample,
            feature_names=feature_names
        )
        
        shap_values = explainer(X_sample)

        if len(shap_values.shape) == 3:
            shap_values = shap_values[..., 1]
            
        mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
        top_indices = np.argsort(mean_abs_shap)[-n_top:][::1]
        
        result_df = pd.DataFrame({
            'Признак': [feature_names[i] for i in top_indices],
            'Среднее |SHAP|': [mean_abs_shap[i] for i in top_indices],
            'Влияние': ['Положительное' if np.mean(shap_values.values[:, i]) > 0 
                       else 'Отрицательное' for i in top_indices]
        })


        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        bars = plt.barh(result_df['Признак'], result_df['Среднее |SHAP|'], color='skyblue')
        plt.title(f'Топ-{n_top} важных признаков')
        plt.xlabel('Среднее абсолютное SHAP значение')
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.001, bar.get_y() + bar.get_height()/2, 
                    f'{width:.4f}', va='center')

        plt.subplot(1, 2, 2)
        shap.summary_plot(shap_values, X_sample, 
                         feature_names=feature_names,
                         max_display=n_top,
                         plot_type='dot',
                         show=False)
        plt.title('Распределение SHAP значений')
        plt.gca().set_xlabel('SHAP value (влияние на выход модели)')
        
        plt.tight_layout()
        plt.show()
        
        return result_df
    
    except Exception as e:
        print(f"Ошибка при анализе SHAP: {str(e)}")
        return None
    
analyze_top_shap_features(model, X_test_processed, n_top=5)

In [ ]:
dtp_counts = df_net_3[df_net_3['vehicle_age'] == 1].groupby('vehicle_age').size()

all_levels = df_net_3['vehicle_age'].unique()
dtp_counts = dtp_counts.reindex(all_levels, fill_value=0)

dtp_counts = dtp_counts.sort_values(ascending=False)

plt.figure(figsize=(12, 6))
dtp_counts.plot(kind='bar', color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Возраст автомобиля')
plt.ylabel('Количество ДТП')
plt.title('Зависимость количества ДТП от возраста автомобиля')
plt.tight_layout()
plt.show()

Итак, можно увидеть что Возраст авто (VEHICLE_AGE) в значении 1 год является главной причной ДТП.

### Вывод по зависимости и вкладам признаков

Возраст авто - является самым многочисленным по причине ДТП. Соответственно, наша модель верна.

**Возраст авто** Можно сказать что для предотвращения ДТП от длительности эксплуатации спасает обычное техническое обслуживание и проверка. При любом нештатном состоянии следует исправлять и машину клиенту не выдавать.

**Контрольное оборудование** Аналогично предыдущему. Контрольное оборудование необходимо проверять на его исправность и корректность выдаваемых данных устройством.

**Регион где произошло ДТП** Путем статистического анализа, который может собирать каршеринговая компания по регионам ее работы, можно сделать вывод какие регионы опасны, например для малоопытных водителей, или вообще опасны. Где риск вероятного ДТП велик и какие факторы в этом замешаны именно в данных регионах. Например, регионы с извилистым серпантином или больше города с интенсивных движением повышают вероятность ДТП.

**Погода** Мокрое покрытие, туман, сильный дождь. Обычный прогноз погоды позволит не выдавать машину в особо опасных погодных условиях.

Посмотрим на связь важного признака с целевой переменной.

In [ ]:
df['vehicle_age'] = df['vehicle_age'].replace(161, df['vehicle_age'].mode()[0])
sobriety_fault_table = pd.crosstab(df['vehicle_age'], df['at_fault'], normalize='index')
print("Доля at_fault для каждого значения vehicle_age:")
print(sobriety_fault_table)

sobriety_fault_table[1].plot(kind='barh', figsize=(10, 6), color='salmon')
plt.xlabel('Доля совершивших ДТП (at_fault = 1)', fontsize=12)
plt.ylabel('Значение признака vehicle_age', fontsize=12)
plt.title('Влияние признака vehicle_age на вероятность совершить ДТП', fontsize=14)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Итоговые выводы

Нашей целью являлось создание модели нейросети которая могла бы оценить риск ДТП по выбранному маршруту движения.

Для достижения цели мы выполнили следующие задачи


- Провели загрузку данных через запросы SQL;
- Провели предобработку данных;
- Исследовательский анализ данных с изображением графиков;
- Для рабочей группы специалистов сформировали 6 вопросов и ответили на 2 из них графически в том числе;
- Сделали не менее 3-х моделей предсказания с перебором разных гиперпараметрами;
- Выбрали одну лучшую модель;
- Оценили вклад каждого признака в модели в ДТП;
- На основе модели исследовали основные факторы ДТП;

Теперь сформируем вывод с некоторыми рекомендациями.

Вся база разделена на 4 разных датафрейма.

case_ids - объединяющая таблица
collisions — общая информация о ДТП
parties — информация об участниках ДТП
vehicles — информация о пострадавших машинах

После создания общего датафрейма мы сделали отбор признаков наиболее сильно влияющих на ДТП, с обоснованием выбор, всего 15 признаков.

Каждый признак был исследован графически и с помощью встроенных методов pandas.

Мы решили делать 4 модели с перебором гиперпараметров и выбрать лучшую. У каждой модели будет своя архитектура и набор параметров.

Использовали валидацию (n_splits=5, shuffle=True, random_state=42) на train данных, чтобы обучить модели. 

Для всех моделей использовали следующие обработчики:
- кодировщик TargetEncoder;
- скалер StandardScaler;

Первая модель:
- 5 слоев, 128, 64, 32, 16, 1 нейронов;
- BatchNorm1d;
- Функции активации везде LeakyReLU, последняя Sigmoid;
- 100 эпох;
- Оптимизатор Адам;
- Размеры батча 128, 256;
- learning_rate для перебора.

Вторая модель:
- 4 слоz, 256, 128, 64, 1нейронов;
- BatchNorm1d и Dropout;
- Функции активации LeakyReLU, ReLU, последняя Sigmoid;
- 100 эпох;
- Оптимизатор Адам;
- Размеры батча 128, 256;
- dropout для перебора.

Третья модель:
- классическая LogisticRegression
Прогон по спискам:
- solver_list = ['lbfgs', 'liblinear', 'saga']
- max_iter_list = [50, 100, 150]
- penalty_list = ['l2', None] 

Четвертая модель:
- RandomForestClassifier
Прогон по спискам:
- n_estimators_list = [50, 80, 150]
- max_depth_list = [3, 5, None]
- min_samples_leaf_list = [1, 2, 3]
- max_features_list = ['sqrt', 'log2']


Данные по моделям собрали в таблицу, со следующими показателями метрики.


Net (Нейросеть) 	    0.6103 	0.0048 	batch_size=128, lr=1e-05

Net_2 (Нейросеть 2) 	0.6082 	0.0054 	batch_size=256, dropout=0.0

Random Forest 	        0.5826 	0.0038  n_est=150, max_d=5, min_leaf=1, max_feat=sqrt

Logistic Regression 	0.5664 	0.0041 	solver=lbfgs, max_iter=50, penalty=None

Модель предсказала 6610 случаев когда ДТП не случится и оно не случилось. 5201 было предсказано как ошибочное, она указала как ДТП не произойдет, а на самом деле ДТП произошло. 1890 модель предсказала что ДТП произойдет и оно произошло. Всего 361 ДТП было предсказано, но оно не произошло. 83% предсказанных ДТП действительно произошли и всего 26% было выявлено. Среднее между ними составляет 40%. Всего модель верно попала в 60% случаев.
Модель великолепно находит случаи, когда ДТП не произошло.
Ложных срабатываний всего 5%.

Можно увидеть что Возраст авто (VEHICLE_AGE) в значении 1 год является главной причной ДТП.

Доля at_fault для каждого значения vehicle_age:
       
Возраст авто    вероятность ДТП 
                  0         1
- 0.0          0.560091  0.439909
- 1.0          0.578389  0.421611
- 2.0          0.435681  0.564319

Еще раз кратко опишем лучшую модель:
- кодировщик OneHotEncoder;
- скалер StandartScaler;
- 4 слоя, 256, 128, 64, 1 нейронов;
- BatchNorm1d и Dropout;
- Функции активации LeakyReLU, ReLU, последняя Sigmoid;
- 100 эпох;
- Оптимизатор Адам;
- Размеры батча 128, 256;
- dropout для перебора.

**Результат работы модели по метрике ROC_AUC 0.6103** 

Обычная техническая проверка автомобиля, в силу его возраста, позволит избежать множество ДТП.

Для улучшения модели можно посоветовать собирать статистику по следующим факторам:
- Из состояния авто факторы про:
  - тип резины и его износ;
  - состояние тормозной системы;
  - работоспособность систем освещения;
  - состояние рулевой системы и задних зеркал;
- Наличие (1/0) животной фауны в местах указанных знаком;
- Наличие (1/0) ж/д путей в зонах эксплуатации авто;
- Наличие (1/0) сильного бокового ветра;
- Здоровье водителя (дальтонизм, тугоухость, слабое зрение и др.);
- Криминальное прошлое клиента (собирать по возможности от сотрудников полиции);

Небольшие рекомендации заказчику по созданию адекватной системы оценки риска при выдаче авто:
- Главный фактор ДТП - трезвость. Выдавать авто нужно только при проверке истории страховых случаев данного водителя. Если причиной ДТП является состояние алкогольного (и еще наркотического) опьянения, то машину нужно выдавать на более суровых и дорогих условиях;
- При учете других факторов нужно опираться на опыт водителя, его семейное положение (семейные как правило водят аккуратно), наличие некоторых зрительных и слуховых отклонений. Понятно, что права на вождение выдаются уже с учетом этого, но каршерингу лучше обратить еще раз внимание на дальтоников и слабослышащих;
- Анкетирование: задавать вопрос - какие марки алкоголя предпочитает клиент.